In [1]:
# script intended to compare the two csv's produced from final_run_statistics.py
# located in Database\report, named final_run_statistics.csv and vmt_statistics.csv

In [2]:
import os
import numpy as np, pandas as pd
import re

runs = [
    r'E:\tko\AdHoc\20230927_TBMTesting\c23q2_700_RSP57_selectline',
    r'E:\tko\AdHoc\20230321_SelectLinkTest\c22q2_100_20220807_SelectLink_setup',
    r'M:\proj1\tko\AdHoc\2023-07_Conformity Macros\c23q2_700_20230807'
]

#potential parameter --
#if you want to save in a different location than the first run in the list, enter it here:
save_location=None

#clean up inputs if entered with slashes at end, and make save_location into xlsx output file
for run in runs: 
    while run.endswith('\\') or run.endswith('/'):
        run = run[:-1]
if save_location:
    while save_location.endswith('\\') or save_location.endswith('/'):
        save_location = save_location[:-1]
    save_location = f'{runs[0]}\\run_statistics_comparison.xlsx'
else:
    save_location = f'{runs[0]}\\cmap_trip-based_model\\Database\\report\\run_statistics_comparison.xlsx'


xlsx = pd.ExcelWriter(save_location)  #<<-- creation of excelwriter object for pandas to write worksheets


# ---
print(f'Begin model comparison. \nReport will save to: {save_location}')
# ---

vmt_dfs = []
run_dfs = []

for run in runs: 
    vmtcsv = f'{run}\\cmap_trip-based_model\\Database\\report\\vmt_statistics.csv'
    runcsv = f'{run}\\cmap_trip-based_model\\Database\\report\\final_run_statistics.csv'

    vmt_df = pd.read_csv(vmtcsv)
    run_df = pd.read_csv(runcsv)

    vmt_dfs.append(vmt_df)
    run_dfs.append(run_df)

mergenumber = len(vmt_dfs) - 1

vmt_basecol = vmt_dfs[0].columns[-1] #name of value column of baseline model run
run_basecol = run_dfs[0].columns[-1] #name of value column of baseline model run

while mergenumber != 0:
    vmt_master = pd.merge(vmt_dfs[0], vmt_dfs[1], how='left')
    comp_col = vmt_dfs[1].columns[-1] #name of value column in comparison model run
    vmt_master[f'diff_{comp_col}'] = vmt_master[comp_col] - vmt_master[vmt_basecol]
    vmt_master[f'pctdiff_{comp_col}'] = vmt_master[f'diff_{comp_col}'] / vmt_master[vmt_basecol]
    vmt_dfs = vmt_dfs[2:]
    vmt_dfs.insert(0, vmt_master)
    mergenumber -= 1
vmt_master.set_index(keys=['Geography','Road Type'], inplace=True)


tuples = []
for column in vmt_master.columns.tolist():
    if column==vmt_basecol:
        tuples.append((column, 'Baseline Value'))
    elif column.startswith('diff'):
        tuples.append((column[5:], 'Diff'))
    elif column.startswith('pctdiff'):
        tuples.append((column[8:], 'Pct Diff'))
    else:
        tuples.append((column, 'Value'))
vmt_master.columns = pd.MultiIndex.from_tuples(tuples)


mergenumber = len(run_dfs) - 1
while mergenumber != 0:
    run_master = pd.merge(run_dfs[0], run_dfs[1], how='left')
    comp_col = run_dfs[1].columns[-1] #name of value column in comparison model run
    run_master[f'diff_{comp_col}'] = run_master[comp_col] - run_master[run_basecol]
    run_master[f'pctdiff_{comp_col}'] = 100 * run_master[f'diff_{comp_col}'] / run_master[run_basecol]
    run_dfs = run_dfs[2:]
    run_dfs.insert(0, run_master)
    mergenumber -= 1
run_master.set_index(keys=run_master.columns.tolist()[0:2], inplace=True)


tuples=[]
for column in run_master.columns.tolist():
    if column==run_basecol:
        tuples.append((column, 'Baseline Value'))
    elif column.startswith('diff'):
        tuples.append((column[5:], 'Diff'))
    elif column.startswith('pctdiff'):
        tuples.append((column[8:], 'Pct Diff'))
    else:
        tuples.append((column, 'Value'))
run_master.columns = pd.MultiIndex.from_tuples(tuples)


run_master.to_excel(xlsx, sheet_name='Final_Run_Stats')
vmt_master.to_excel(xlsx, sheet_name='VMT_Stats')
xlsx.close()

print('Done.')

Begin model comparison. 
Report will save to: E:\tko\AdHoc\20230927_TBMTesting\c23q2_700_RSP57_selectline\cmap_trip-based_model\Database\report\run_statistics_comparison.xlsx
Done.
